## ReAct agent:
It is a type of the agentic ai architecture where the agent performs the loop as:-

 (Reasoning ---> Action ---> Observe ---> Reasoning)

Instead of making a plan(sequence of task) to achieve the end goal, in ReAct architecture, we do the reasoning step by step due to which we go from several loops over the brain and the tools depending upon the question and the question complexity.

When the agent gets the question, 

"what is the capital city of Nepal and what is the population of the capital city?" then,
It doesn't make the plan to reach the goal, what is does is it first reasons what is the capital city of Nepal and then after the result is accquired, it then finds the population of the result obtained from previous reasoning and in between it calls required tools and observes the answer from the tool.

Like the other Agent, this Agent is binded with tools too and makes dynamic tool calls if the appropriate tool is avaiable like TavilySearch, bing, Arxiv, Wikipidea etc. 

This is the base of almost all agentic system and basic architecture for agentic system req uiring the single-agent.

ReAct is less powerful and more costly than planner-executer architecture as we have to perform reasoning in the every step where as the planner-executer agent does it ones and executes it step-by-step as plan without invoking llm(brain) for next step which is more costly and high bills.




In [ ]:
from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage, ToolMessage, AIMessage
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from typing import Annotated, TypedDict, Literal
from dotenv import load_dotenv
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_tavily import TavilySearch
import os

In [ ]:
load_dotenv()

In [ ]:
llm = ChatGroq(
    model=os.getenv("model"),
    api_key=os.getenv("api_key")
)

In [ ]:
# state
import operator
class Messagestate(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]
    plan : str

class Plan(BaseModel):
    plan : str = Field(description="the plan should in string format in point wide form")

PlanFormat = PydanticOutputParser(pydantic_object=Plan)

In [ ]:
arxiv_wallpaper = ArxivAPIWrapper(top_k_results= 1, doc_content_chars_max= 400)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wallpaper)

wikipedia_wallpaper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max= 150 )
wikipedia_tool = WikipediaQueryRun(api_wrapper=wikipedia_wallpaper)

tavily_tool = TavilySearch(max_results=1, topic='general', include_answer=True)

tavily_tool.invoke({'query' : "What are the latest breakthroughs in AI for 2026?"})

In [ ]:
print(wikipedia_tool.invoke("Criastiano Ronaldo"))

In [ ]:
from langchain.tools import tool

In [ ]:

# Define tools
@tool
def multiply(a: int, b: int)->int:
    """used to Multiply two given number `a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """used to Adds two given number`a` and `b`.

    Args:
        a: First int
        b: Second int
    """
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """used to Subtract two given number a and b. 
    Args: 
    a: First int b: Second int
    """
    return a - b

@tool
def divide(a: int, b: int) -> float:
    """used to Divide two given number a and b.
    Args:
        a: First int
        b: Second int
    """
    return a / b

@tool
def root_tool(a: float, b : float, c : float)-> str:
    '''used to solve an quadratic quation given the 3 parametes, a, b and c where the eauipon is ax^2+bx+c=0 and return the value of x'''
    discrimination =  b**2 - (4*a*c)
    val = discrimination
    if val < 0:
        return "There are imaginary root"
    elif val == 0:
        root = (-b)/(2*a)
        return f"The root is {root} and {root}"
    else:
        root1 = (-b + discrimination**0.5)/(2*a)
        root2 = (-b - discrimination**0.5)/(2*a)
        return {'result' : f"The roots are : {root1} and {root2}"}


# Augment the LLM with tools
tools = [add, subtract, multiply, divide, arxiv_tool, wikipedia_tool, tavily_tool, root_tool]
tools_by_name = {tool.name: tool for tool in tools}
llm_with_tools = llm.bind_tools(tools)
print(tools_by_name)

In [ ]:
from langgraph.graph import MessagesState


# Nodes
def llm_call(state: MessagesState):
    """LLM decides whether to call a tool or not"""

    return {
        "messages": [
            llm_with_tools.invoke(state["messages"])
        ],
    }

def tool_node(state: dict):
    """Performs the tool call"""

    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
    return {"messages": result}


# Conditional edge function to route to the tool node or end based upon whether the LLM made a tool call
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Decide if we should continue the loop or stop based upon whether the LLM made a tool call"""

    messages = state["messages"]
    last_message = messages[-1]

    # If the LLM makes a tool call, then perform an action
    if last_message.tool_calls:
        return "tool_node"

    # Otherwise, we stop (reply to the user)
    return END

checkpointer = MemorySaver()
# Build workflow
agent_builder = StateGraph(MessagesState)

# Add nodes
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tool_node", tool_node)

# Add edges to connect nodes
agent_builder.add_edge(START, "llm_call")
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    ["tool_node", END]
)
agent_builder.add_edge("tool_node", "llm_call")
# agent_builder.add_edge("tool_node", "llm_call")

# Compile the agent
agent = agent_builder.compile(checkpointer=checkpointer)

# Show the agent
# display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

# Invoke
messages = [HumanMessage(content="Who made Python language and multiply his age with 4")]
config = {'configurable' : {'thread_id' : 2}}
messages = agent.invoke({"messages": messages}, config=config)
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
agent

In [ ]:
msg = agent.get_state(config=config).values
msg


In [ ]:
for m in msg['messages']:
    if isinstance(m, HumanMessage):
        print('Human')
        print(m.content)
    elif isinstance(m, AIMessage):
        print('\nAI')
        # print(m.additional_kwargs['reasoning_content'])
        print(m.content)
        print(m.tool_calls)
    else:
        print("\ntool")
        print(m)
    print('='*100)

In [ ]:
for m in agent.get_state(config=config).values['messages']:
    print(m)

In [ ]:
print(agent.get_state(config=config).values['messages'][-1].content)

In [ ]:
messages = [HumanMessage(content="give me the recent AI new and then multiply 48459594144 by 29854854824545")]
config = {'configurable' : {'thread_id' : 3}}
messages = agent.invoke({"messages": messages}, config=config)
for m in messages["messages"]:
    m.pretty_print()